In [1]:
import sys
sys.path.append('..')
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import torchvision.transforms as tf

from src.models.twins.model import TwinsUNETR
from src.models.train import Trainer
from src.features.segmentation.dataset import SegmentationDataset

from transformers import AutoImageProcessor

In [2]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-tiny-patch4-window16-256")
image_processor.do_resize = False
image_processor.do_rescale = False
model = TwinsUNETR()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [3]:
twins = TwinsUNETR()

In [4]:
device = torch.device("mps")
twins = twins.to(device)

In [5]:
for name, param in twins.named_parameters():
    if param.device != "mps:0":
        print(param.device)
        break

mps:0


In [6]:
transform = tf.Resize((256, 256))

In [7]:
train_dataset = SegmentationDataset(
    images_dir="../data/split/train/images/",
    masks_dir="../data/split/train/masks/",
    image_transform=transform,
    mask_transform=transform
)

val_dataset = SegmentationDataset(
    images_dir="../data/split/valid/images/",
    masks_dir="../data/split/valid/masks/",
    image_transform=transform,
    mask_transform=transform
)


train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)


In [8]:
for image, target in train_dataloader:
    image = image.to(device)
    target = target.to(device)
    out = twins.train_on_batch(image, target)
    break

torch.Size([2, 64, 64, 64])
torch.Size([2, 128, 32, 32])
torch.Size([2, 256, 16, 16])
torch.Size([2, 512, 8, 8])


IndexError: index 1 is out of bounds for dimension 0 with size 0